# Задание 1. Прямые методы решения линейных систем

In [3]:
from latexifier import latexify
from IPython.display import display, Markdown
import numpy as np
from typing import Callable, List
import warnings
warnings.filterwarnings('ignore')

In [4]:
class Matrix(np.ndarray):
    """extends np.ndarray"""

    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        return obj

    def inverse_rows(self, i: int, j: int):
        """Swaps i-th and j-th rows of a matrix"""
        self[i], self[j] = self[j].copy(), self[i].copy()

    def inverse_cols(self, i: int, j: int):
        """Swaps i-th and j-th columns of a matrix"""
        self = self.transpose()
        self.inverse_rows(i, j)
        self = self.transpose()
    
    def tex(self) -> str:
        """Latexifies the matrix rounding up values up to 6 decimal values"""
        return latexify(self)

def append(M: np.ndarray, N: np.ndarray) -> Matrix:
    """Appends two numpy arrays with axis=1"""
    return Matrix(np.append(arr=np.asarray(M), values=N, axis=1))

def print_tex(*argv):
    """Displays a LaTex Markdown output"""
    res = ' '.join(['$$'] + [arg for arg in argv] + ['$$'])
    display(Markdown(res))

def get_latex_column(arg: str, n: int) -> str:
    """Prepares a string with LaTex syntax for a columt of {arg} with indeces form 1 to {n}"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = arg + "_{" + str(i+1) + "}" + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)


In [5]:
s = get_latex_column("x", 5)
print_tex(s, s)

$$ \begin{pmatrix}
x_{1}\\
x_{2}\\
x_{3}\\
x_{4}\\
x_{5}
\end{pmatrix} \begin{pmatrix}
x_{1}\\
x_{2}\\
x_{3}\\
x_{4}\\
x_{5}
\end{pmatrix} $$

In [8]:
n = int(input()) # Размерность матрицы A
A = Matrix(np.random.rand(n, n) * 10)
B = Matrix(np.random.rand(n, 1) * 10)
print_tex('A = ', A.tex(), ',~~', 'B = ', B.tex())
print_tex(A.tex(), get_latex_column('x', n), '=', B.tex())

$$ A =  \begin{pmatrix} 6.327071 & 8.743031 & 6.274022 & 7.278662 \\ 5.725189 & 8.179226 & 3.446954 & 2.075253 \\ 7.476220 & 3.153339 & 2.272707 & 7.062988 \\ 8.234709 & 6.151125 & 7.360045 & 1.590035 \end{pmatrix} ,~~ B =  \begin{pmatrix} 4.513907 \\ 3.705768 \\ 1.479898 \\ 5.155648 \end{pmatrix} $$

$$ \begin{pmatrix} 6.327071 & 8.743031 & 6.274022 & 7.278662 \\ 5.725189 & 8.179226 & 3.446954 & 2.075253 \\ 7.476220 & 3.153339 & 2.272707 & 7.062988 \\ 8.234709 & 6.151125 & 7.360045 & 1.590035 \end{pmatrix} \begin{pmatrix}
x_{1}\\
x_{2}\\
x_{3}\\
x_{4}
\end{pmatrix} = \begin{pmatrix} 4.513907 \\ 3.705768 \\ 1.479898 \\ 5.155648 \end{pmatrix} $$

### Решение методом Гаусса единственного деления

In [9]:
def SingleGauss(A: Matrix, B: Matrix, n: int) -> Matrix:
    A_ = A.copy()
    B_ = B.copy()
    X = [[0] for _ in range(n)]
    for k in range(n):
        for j in range(k+1, n):
            d = A_[j][k] / A_[k][k]
            for i in range(k, n):
                A_[j][i] = A_[j][i] - d * A_[k][i]
            B_[j] -= d * B_[k][0]
    for k in range(n-1, -1, -1):
        d = 0
        for j in range(k, n):
            s = A_[k][j] * X[j][0]
            d += s
        X[k][0] = (B_[k][0] - d) / A_[k][k]
    return Matrix(np.array(X))


X = SingleGauss(A, B, n)
B_ = np.matmul(A, X)
print_tex('X^* = ', get_latex_column("x^*", n), '=', X.tex())
print_tex(r'A \times X^* = ', A.tex(), X.tex(), '=', B_.tex(), '= B^*')
print_tex('B = ', B.tex(), r'\stackrel{?}{=}', B_.tex(), '= B^*')
print_tex(r'|B - B^*| = ', str(float(max(abs(B - B_)))))


$$ X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}
\end{pmatrix} = \begin{pmatrix} 0.071577 \\ 0.248118 \\ 0.438565 \\ -0.118131 \end{pmatrix} $$

$$ A \times X^* =  \begin{pmatrix} 6.327071 & 8.743031 & 6.274022 & 7.278662 \\ 5.725189 & 8.179226 & 3.446954 & 2.075253 \\ 7.476220 & 3.153339 & 2.272707 & 7.062988 \\ 8.234709 & 6.151125 & 7.360045 & 1.590035 \end{pmatrix} \begin{pmatrix} 0.071577 \\ 0.248118 \\ 0.438565 \\ -0.118131 \end{pmatrix} = \begin{pmatrix} 4.513907 \\ 3.705768 \\ 1.479898 \\ 5.155648 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix} 4.513907 \\ 3.705768 \\ 1.479898 \\ 5.155648 \end{pmatrix} \stackrel{?}{=} \begin{pmatrix} 4.513907 \\ 3.705768 \\ 1.479898 \\ 5.155648 \end{pmatrix} = B^* $$

$$ |B - B^*| =  2.6645352591003757e-15 $$